In [1]:
import pinocchio as pin
from pinocchio.utils import *

import example_robot_data as robex
import numpy as np
from numpy.linalg import inv, pinv, eig, norm, svd, det
from scipy.optimize import fmin_bfgs
import time
import copy
import hppfcl
import vizutils

import math


In [2]:
robot = robex.loadSolo()

#Viewer = pin.visualize.GepettoVisualizer
Viewer = pin.visualize.MeshcatVisualizer

viz = Viewer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(loadModel=True)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
hasattr(viz.viewer, 'jupyter_cell') and viz.viewer.jupyter_cell()


In [4]:
viz.display(robot.q0)

vizutils.addViewerBox(viz, 'world/box', .2, 2, .4, [1., 1, .2, 1])
vizutils.applyViewerConfiguration(viz, 'world/box', [1, -.7, .2, 1, 0, 0, 0])

vizutils.addViewerBox(viz, 'world/floor', 12, 12, .1, [.8, .8, .8, .41])
vizutils.applyViewerConfiguration(viz, 'world/floor', [0, 0, -.01, 1, 0, 0, 0])


In [5]:
class RootBox:
    def __init__(self):
        vizutils.addViewerBox(viz, 'world/root', .43, .26, .15, [1., .2, .2, .5])
        
    def alpha(self,alph):
        self.alph = alph + 0.0
        vizutils.addViewerBox(viz, 'world/root', .43, .26, .15, [1., .2, .2, alph])

        
    def move(self, q):
        xRoot = q.copy()[:7]
        vizutils.applyViewerConfiguration(viz, 'world/root', xRoot)
        
rootBox = RootBox()

In [6]:
rootBox.alpha(0.5)

In [7]:
rmodel = robot.model
rdata = rmodel.createData()

#for f in robot.model.frames:
#    print(f.name)

In [8]:
# Get index of end effector
#idx = rmodel.getFrameId('HL_FOOT')
#print(idx)



### COST FUNCTIONS:

In [9]:
class Cost3d:
    def __init__(self, rmodel, rdata, frameIndex=None, ptarget=None, viz=None):
        self.rmodel = rmodel
        self.rdata = rdata
        self.ptarget = ptarget if ptarget is not None else  np.array([0.5, 0.1, 0.27])
        self.frameIndex = frameIndex if frameIndex is not None else robot.model.nframes - 1
        self.viz = viz

    def residual(self, q):
        pin.framesForwardKinematics(self.rmodel, self.rdata, q)
        M = self.rdata.oMf[self.frameIndex]
        return M.translation - self.ptarget

    def calc(self, q):
        return sum(self.residual(q) ** 2)

    # --- Callback
    def callback(self, q):
        if self.viz is None:
            return
        pin.framesForwardKinematics(self.rmodel, self.rdata, q)
        M = self.rdata.oMf[self.frameIndex]
        rootBox.move(q)

        self.viz.display(q)
        time.sleep(0.05) #exagerating the time to make it visible

In [10]:
class Cost6d:
    def __init__(self, rmodel, rdata, frameIndex=None, Mtarget=None, viz=None):
        self.rmodel = rmodel
        self.rdata = rdata
        self.Mtarget = Mtarget if Mtarget is not None else pin.SE3(pin.utils.rotate('z', 90),
                                                                   np.array([0,0,.2]))  # x, y, z
        self.frameIndex = frameIndex if frameIndex is not None else robot.model.nframes-1
        
        self.viz = viz

    def residual(self, q):
        '''Compute score from a configuration'''
        pin.forwardKinematics(self.rmodel, self.rdata, q)
        M = pin.updateFramePlacement(self.rmodel, self.rdata, self.frameIndex)
        self.deltaM = self.Mtarget.inverse() * M
        return pin.log(self.deltaM).vector

    def calc(self, q):
        return sum(self.residual(q) ** 2)

    def callback(self, q):
        if self.viz is None:
            return
        self.viz.display(q)
        time.sleep(0.05)

In [25]:
cost = Cost3d(rmodel, rdata, frameIndex = 1, ptarget= robot.q0[:3], viz=viz)
qguess = robot.q0.copy()
qopt = fmin_bfgs(cost.calc, qguess, callback=cost.callback)

rootBox.move(qopt)
viz.display(qopt)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 17
         Gradient evaluations: 1


In [12]:
class CostPosture:
    def __init__(self, rmodel, rdata, qref=None, viz=None):
        # viz, rmodel and rdata are taken to respect the API but are not useful.
        self.qref = qref if qref is not None else pin.randomConfiguration(rmodel)
        self.removeFreeFlyer = robot.model.joints[1].nq == 7  # Don't penalize the free flyer if any.

    def residual(self, q):
        if self.removeFreeFlyer:
            return (q - self.qref)[7:]
        else:
            return q - self.qref

    def calc(self, q):
        return sum(self.residual(q) ** 2)

In [13]:
class SumOfCostFreeFlyer:
    def __init__(self, rmodel, rdata, costs, weights):
        self.rmodel = rmodel
        self.costs = costs
        self.weights = np.array(weights)

    def calc(self, q):
        q = pin.normalize(self.rmodel, q)
        # Alternatively, add this extra term to the sum
        extra = sum((q - pin.normalize(rmodel, q)) ** 2) * 100
        return sum(self.weights * [cost.calc(q) for cost in self.costs])   + extra

    def callback(self, q):
        q = pin.normalize(self.rmodel, q)
        rootBox.move(q)
        for c in self.costs:
            if hasattr(c, 'callback'):
                c.callback(q)
       
        time.sleep(0.05)


# More realistic now:

### create segment object (start, end, orientation)

In [14]:
class Segment:
    def __init__(self, name, startq, endq, size=None):
        self.name   = 'world/segStart_' + str(name)
        self.startq = startq
        self.endq   = endq
        self.size   = size if size is not None else .5
        
        
        vizutils.addViewerSphere(viz, self.name, .02, [.2, 1, .1, 1])
        vizutils.applyViewerConfiguration(viz, self.name, startq)    
        
        
    def getOrientation(self):
        ## see how the orientation relates to the transform 
        ##myradians = math.atan2(targetY-gunY, targetX-gunX)
        theta = math.atan2(self.endq[1]-self.startq[1],self.endq[0]-self.startq[0])
        return theta
        
    def remove(self):
        vizutils.addViewerSphere(viz, self.name, .0002, [.2, 1, .1, 0])

In [15]:
class Objective:
    def __init__(self, name, q):
        self.name   = name if name is not None else 'world/obj'
        self.q      = q
        
        vizutils.addViewerSphere(viz, self.name, .04, [.2, .1, 1, 1])
        vizutils.applyViewerConfiguration(viz, self.name, q)    
    
    def position(self):
        return self.q
    
    def remove(self):
        vizutils.addViewerSphere(viz, self.name, .0002, [.2, 1, .1, 0])

### calculate distance & create list of segment objects

In [16]:
import math

def getDist(start,end):  
    dist = math.hypot(end[0] - start[0], end[1] - start[1])
    return dist
    
def clearPath(segments):
    for seg in segments:
        seg.remove()


In [17]:
def genSegments(start, end, segSize=None):
    #segSize = .3 ## Max step size
    segSize   = segSize if segSize is not None else .3


    distance = getDist(start,end)  
    print ("OG distance: " + str(distance))

    segAmount = math.ceil(distance / segSize)
    print("segments: " + str(segAmount))

    xstep = (end[0] - start[0])/segAmount
    ystep = (end[1] - start[1])/segAmount

    segments = []
    for i in range(segAmount):
        xiTmp = [start[0]+xstep*i,start[1]+ystep*i,.2,1,0,0,0]
        xfTmp = [xiTmp[0]+xstep,xiTmp[1]+ystep,.2,1,0,0,0]
    
        segments.append(Segment(i,xiTmp,xfTmp))
        
    return segments

### travel through the segments until final destination

In [18]:
def travelSegments(segments, start):
    qguess = start
    prevOrientation = 0
    for seg in segments:
        orientation = math.degrees(seg.getOrientation())
    
        orMod = orientation - prevOrientation
        print(orMod)
        MtargetEnd = pin.SE3(pin.utils.rotate('z', 0), np.array(seg.endq[:3]))
        #MtargetEnd = pin.SE3(pin.utils.rotate('z', prevOrientation+orMod), np.array(seg.endq[:3]))

        prevOrientation = orientation
        
        mycost = SumOfCostFreeFlyer(rmodel, rdata,
                             [Cost6d(rmodel, rdata, frameIndex = 1,Mtarget = MtargetEnd, viz=viz),
                                CostPosture(rmodel, rdata, qref = qguess )], [1, 0.001])
        #[Cost3d(rmodel, rdata, frameIndex = 1, ptarget= seg.endq[:3], viz=viz),
    
        qopt = fmin_bfgs(mycost.calc, qguess, callback=cost.callback)
        qguess = qopt
        rootBox.move(qopt)
        viz.display(qopt)
    
    clearPath(segments)
    return qopt

 ### set random final position and go to it 

In [19]:
cost = Cost3d(rmodel, rdata, frameIndex = 1, ptarget= robot.q0[:3], viz=viz)
qguess = robot.q0.copy()
qopt = fmin_bfgs(cost.calc, qguess, callback=cost.callback)

rootBox.move(qopt)
viz.display(qopt)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 17
         Gradient evaluations: 1


In [20]:
rdata.oMi[1]

  R =
1 0 0
0 1 0
0 0 1
  p =     0     0 0.235

In [21]:
import random

def genRandomObjective(name=None):
    name   = 'world/segEnd_' + str(name)

    areaSize = 5
    randX = round(random.uniform(-areaSize, areaSize),2)
    randY = round(random.uniform(-areaSize, areaSize),2)

    xf = [randX, randY, .2, 1, 0, 0, 0]  # x, y, z, quaternion
    
    objRand = Objective(name,xf)
    return objRand


In [22]:
objq = genRandomObjective(1)
segments = []
segments = genSegments(qguess, objq.position())
qguess = travelSegments(segments,qguess)

OG distance: 3.7377132046212425
segments: 13
139.44858559153494
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 51
         Gradient evaluations: 3
0.0
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 51
         Gradient evaluations: 3
0.0
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 51
         Gradient evaluations: 3
0.0
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 51
         Gradient evaluations: 3
0.0
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2
         Function evaluations: 85
         Gradient evaluations: 5
2.842170943040401e-14
Optimization terminated successfully.
         C

### Random list of objectives

In [23]:
targets = []
for i in range(10):
    targets.append(genRandomObjective(i))
    

for tar in targets:
    segments = []
    segments = genSegments(qguess, tar.position())
    qguess = travelSegments(segments,qguess)
    tar.remove()

OG distance: 6.074816879350805
segments: 21
23.991218308273954
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2
         Function evaluations: 68
         Gradient evaluations: 4
0.0
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 4
         Function evaluations: 119
         Gradient evaluations: 7
0.0
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2
         Function evaluations: 68
         Gradient evaluations: 4
0.0
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2
         Function evaluations: 68
         Gradient evaluations: 4
0.0
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 3
         Function evaluations: 102
         Gradient evaluations: 6
0.0
Optimization terminated successfully.
         Current function v

### Fake RRT :)

In [27]:
# define set points

qObj = [.5, .7, .2, 1, 0, 0, 0]  # x, y, z, quaternion   
obj_1 = Objective('1',qObj)

qObj = [1.5, .8, .2, 1, 0, 0, 0]  # x, y, z, quaternion   
obj_2 = Objective('2',qObj)

qObj = [2.3, -1, .2, 1, 0, 0, 0]  # x, y, z, quaternion   
obj_3 = Objective('3',qObj)

qObj = [2, -3, .2, 1, 0, 0, 0]  # x, y, z, quaternion   
obj_4 = Objective('4',qObj)

objectives = [obj_1,obj_2,obj_3,obj_4]

for obj in objectives:
    segments = []
    segments = genSegments(qguess, obj.position())
    qguess = travelSegments(segments,qguess)
    obj.remove()

OG distance: 3.9924929485286547
segments: 14
112.0678995125933
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2
         Function evaluations: 68
         Gradient evaluations: 4
0.0
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 3
         Function evaluations: 102
         Gradient evaluations: 6
0.0
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 4
         Function evaluations: 102
         Gradient evaluations: 6
0.0
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 4
         Function evaluations: 102
         Gradient evaluations: 6
0.0
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 4
         Function evaluations: 102
         Gradient evaluations: 6
0.0
Optimization terminated successfully.
         Current function